In [45]:
import time
import json
import os
import glob
import itertools
from operator import itemgetter
from kafka import KafkaProducer
from kafka import KafkaConsumer
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession


In [46]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files


In [47]:
files = get_files('meta_data')
producer = KafkaProducer(bootstrap_servers =
  ['localhost:9092'],value_serializer=lambda v: json.dumps(v).encode('utf-8'))


In [48]:
for file in files:
    with open(file, 'r') as f:
        data = json.load(f)
        producer.send('input_topic', value = data, key =json.dumps(data['userId']).encode('utf-8'))


In [49]:
consumer = KafkaConsumer ('input_topic',bootstrap_servers = ['localhost:9092'],
value_deserializer=lambda m: json.loads(m.decode('utf-8')),consumer_timeout_ms=1000,auto_offset_reset='earliest')

In [50]:
kafka_data =[]
for message in consumer:
    kafka_data.append(message.value)

In [64]:
producer2 = KafkaProducer(bootstrap_servers =
  ['localhost:9092'],value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [65]:
sorted_data = sorted(kafka_data,key=itemgetter('userId'))


for key, group in itertools.groupby(sorted_data, key=lambda x:x['userId']):
   
    arr = list(group)
    big = -1
    small = float('inf')
    for obj in arr :
       
        sent_at = obj['metadata']['sentAt']
        recive_at = obj['metadata']['receivedAt']
        time = sent_at+recive_at
        if time < small:
            small = time
        if time > big :
            big = time
    dict = {'userId':key,'firstSeen': small,'lastSeen':big}
    producer2.send('output_topic', value = dict, key =json.dumps(key).encode('utf-8'))
        

In [68]:
consumer = KafkaConsumer ('output_topic',bootstrap_servers = ['localhost:9092'],
value_deserializer=lambda m: json.loads(m.decode('utf-8')),consumer_timeout_ms=1000,auto_offset_reset='earliest')

In [69]:
for message in consumer:
    kafka_data.append(message)